In [1]:
library(tidyverse)
library(stringr)
library(ggplot2)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
spectrum_generator <- function(count=100,coverage=2/3,offset=0,reverse=FALSE) {
    tau = 2*pi
    color = vector('list',3)
    coverage = ((-1)^reverse)*coverage
    offset = ((-1)^reverse)*offset
    for (iter in 1:3) {
        values = c()
        theta = (offset + iter - 1) * tau/3
        for (x in seq(0,tau*coverage,tau*coverage/(count-1))[1:count]) {
            values = c(values, cos(x + theta)/2 + 0.5)
        }
        color[[iter]] = values
    }
    return(rgb(color[[1]],color[[2]],color[[3]]))
}

In [3]:
statsfile_path = "PD.sumstats"

In [4]:
gwas_raw <- read.csv(statsfile_path, sep="\t")

In [5]:
names(gwas_raw)[names(gwas_raw) == "POS"] <- "BP"
gwas_raw$LOG10P <- -log10(gwas_raw$P)

In [6]:
head(gwas_raw)

,SNP,CHR,BP,A1,A2,REF,EAF,Beta,se,P,N,INFO,LOG10P
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
1,rs10399793,1,49298,T,C,T,0.37621700,-0.000281739,0.000363748,0.44,459256,0.342797,0.35654732
2,rs2462492,1,54676,C,T,C,0.59941400,0.000113573,0.000360334,0.75,459256,0.340158,0.12493874
3,rs3107975,1,55326,T,C,T,0.99155100,0.003083870,0.002005890,0.12,459256,0.324228,0.92081875
4,rs74447903,1,57033,T,C,T,0.99822100,-0.003113990,0.004464200,0.49,459256,0.296256,0.30980392
5,1:70728_C_T,1,70728,C,T,C,0.99783400,-0.000368448,0.003631370,0.92,459256,0.365713,0.03621217
6,rs2462495,1,79033,A,G,A,0.00129077,-0.005263570,0.004622200,0.25,459256,0.536566,0.60205999


In [7]:
gwas_data <- gwas_raw

In [8]:
don <- gwas_data %>% 
  
  # Compute chromosome size
  group_by(CHR) %>% 
  summarise(chr_len=max(BP)) %>% 
  
  # Calculate cumulative position of each chromosome
  mutate(tot=cumsum(as.numeric(chr_len))-chr_len) %>%
  select(-chr_len) %>%
  
  # Add this info to the initial dataset
  left_join(gwas_data, ., by=c("CHR"="CHR")) %>%
  
  # Add a cumulative position of each SNP
  arrange(CHR, BP) %>%
  mutate(BPcum=BP+tot)

axisdf <- don %>% group_by(CHR) %>% summarize(center=( max(BPcum) + min(BPcum) ) / 2 )

spectrum <- spectrum_generator(
    count = 22,
    coverage = 5/6,
    offset = -0.25,
    reverse = TRUE
)

manplot <- ggplot(don, aes(x=BPcum, y=LOG10P)) +
    
    # Show all points
    geom_point( aes(color=as.factor(CHR)), alpha=1, size=0.2) +
    scale_color_manual(values = spectrum) +
    
    # custom X axis:
    scale_x_continuous( label = axisdf$CHR, breaks= axisdf$center ) +
    scale_y_continuous(expand = c(0, 0) ) +     # remove space between plot area and x axis
  
    # Custom theme:
    theme_bw() +
    theme( 
      legend.position="none",
      panel.border = element_blank(),
      panel.grid.major.x = element_blank(),
      panel.grid.minor.x = element_blank()
    )

In [9]:
ggsave('PD_manplot.png', plot=manplot, width=24, height=8, units='cm', dpi=600 )

In [20]:
gwas_data <- gwas_raw[abs(gwas_raw$Beta) > gwas_raw$se,]

In [23]:
head(gwas_data)

,SNP,CHR,BP,A1,A2,REF,EAF,Beta,se,P,N,INFO,LOG10P
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
3,rs3107975,1,55326,T,C,T,0.99155100,0.00308387,0.00200589,0.12,459256,0.324228,0.9208188
6,rs2462495,1,79033,A,G,A,0.00129077,-0.00526357,0.00462220,0.25,459256,0.536566,0.6020600
14,rs576404767,1,544584,C,T,C,0.99811000,0.00336628,0.00328980,0.31,459256,0.492517,0.5086383
19,rs538567606,1,565196,T,C,T,0.99769000,0.00447451,0.00349176,0.20,459256,0.369036,0.6989700
37,rs375217967,1,568800,G,A,G,0.98306800,0.00160406,0.00138525,0.25,459256,0.328855,0.6020600
50,rs544671234,1,705942,A,T,A,0.99664100,0.00246894,0.00226337,0.28,459256,0.607378,0.5528420


In [24]:
don <- gwas_data %>% 
  
  # Compute chromosome size
  group_by(CHR) %>% 
  summarise(chr_len=max(BP)) %>% 
  
  # Calculate cumulative position of each chromosome
  mutate(tot=cumsum(as.numeric(chr_len))-chr_len) %>%
  select(-chr_len) %>%
  
  # Add this info to the initial dataset
  left_join(gwas_data, ., by=c("CHR"="CHR")) %>%
  
  # Add a cumulative position of each SNP
  arrange(CHR, BP) %>%
  mutate(BPcum=BP+tot)

axisdf <- don %>% group_by(CHR) %>% summarize(center=( max(BPcum) + min(BPcum) ) / 2 )

spectrum <- spectrum_generator(
    count = 22,
    coverage = 5/6,
    offset = -0.25,
    reverse = TRUE
)

manplot <- ggplot(don, aes(x=BPcum, y=LOG10P)) +
    
    # Show all points
    geom_point( aes(color=as.factor(CHR)), alpha=1, size=0.2) +
    scale_color_manual(values = spectrum) +
    
    # custom X axis:
    scale_x_continuous( label = axisdf$CHR, breaks= axisdf$center ) +
    scale_y_continuous(expand = c(0, 0) ) +     # remove space between plot area and x axis
  
    # Custom theme:
    theme_bw() +
    theme( 
      legend.position="none",
      panel.border = element_blank(),
      panel.grid.major.x = element_blank(),
      panel.grid.minor.x = element_blank()
    )

In [25]:
ggsave('PD_manplot_reduced.png', plot=manplot, width=24, height=8, units='cm', dpi=600 )